In [ ]:
!pip install -q chromadb pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.4 MB/s eta 0:00:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Mumbai-Hackathon

/content/drive/MyDrive/Mumbai-Hackathon


In [111]:
from vertexai.preview.generative_models import GenerativeModel, Part
from google.cloud import storage
from vertexai.vision_models import Image, MultiModalEmbeddingModel, MultiModalEmbeddingResponse
import os
import vertexai
from PIL import Image as PILImage
import chromadb
from chromadb import Client, Documents, Embeddings, EmbeddingFunction
from tqdm import tqdm
import requests
from io import BytesIO
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
text_emb_model = TextEmbeddingModel.from_pretrained("text-embedding-004")

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/hackathon-mumbai/aiml_creds.json"

In [ ]:
vertexai.init(project="ai-coe-386307", location="us-central1")

In [162]:
PROJECT_ID = ""
LOCATION = "us-central1"
model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding")
text_emb_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
client = chromadb.PersistentClient(path="vdb")
collection = client.get_or_create_collection(name="fashion_db")

In [118]:
def generate_embeddings(image):
  response = requests.get(image)
  with open("temp_image.jpg", "wb") as file:
    file.write(response.content)

  image = Image.load_from_file("temp_image.jpg")
  embeddings = model.get_embeddings(
      image=image,
      dimension=1408,
  )
  os.remove("temp_image.jpg")
  return embeddings.image_embedding

def generate_embeddings_path(image):

  image = Image.load_from_file(image)
  embeddings = model.get_embeddings(
      image=image,
      dimension=1408,
  )
  return embeddings.image_embedding

def generate_embeddings_text(text, task):
  return TextEmbeddingInput(text, task)


def image_search(image):
  emb = generate_embeddings_path(image)
  print(emb)
  result = collection.query(
      query_embeddings = emb,
      n_results = 5
  )
  return result


In [ ]:
from pymongo import MongoClient

db_password = ""
connection_string = f""

client = MongoClient(connection_string)

db = client['closet']

collection = db['data_all_1']
documents = collection.find()

prod_li = []
for doc in documents:
  prod_li.append(doc)

In [37]:
file_li = []
emb_li = []
from tqdm import tqdm
for i in tqdm(range(0,len(prod_li),1)):
  prod = prod_li[i]
  for img in prod.get("images"):
    emb = generate_embeddings(img)
    file_li.append(img)
    emb_li.append(emb)

100%|██████████| 313/313 [13:31<00:00,  2.59s/it]


In [50]:
final_li = []
final_emb = []
for i in range(len(file_li)):
  if file_li[i] not in final_li:
    final_li.append(file_li[i])
    final_emb.append(emb_li[i])

In [52]:
collection.add(
    ids = final_li,
    embeddings = final_emb
)

In [54]:
!ls

4d703406-3b54-4053-9bee-67ea09057a02_1.jpg  aiml_creds.json  vdb


In [164]:
test = "/content/drive/MyDrive/Mumbai-Hackathon/1716543492_6546785.webp"
res = image_search(test)

[0.00202352763, 0.0105905877, 0.083145231, 0.0343205296, 0.016351752, 0.0166178085, -0.00450315699, 0.0116493553, -0.00551301427, 0.00507321535, -0.0191380251, 0.0212064087, -0.0110029411, 0.0711062625, -0.0075693354, -0.00580310775, 0.0289753787, 0.0150695536, 0.00257972907, -0.00398789765, 0.0820100307, -0.0352460332, -0.0248368345, 0.0361529477, -0.0402505621, 0.0192859247, 0.0212264638, -0.00948293135, 0.00222051167, -0.0328875557, 0.0201734118, 0.0259642098, 0.00743369618, 0.084982343, 0.028597651, 0.00339480536, 0.0220414046, 0.00697327, 0.00983496662, -0.0280367546, 0.0151790269, -0.0172483157, 0.0261746757, 0.019016888, -0.0406882912, -0.0297825467, 0.00566800544, 0.00341842603, 0.00895892736, -0.0204981156, 0.00635668216, -0.0121516204, -0.017702464, 0.00403061695, -0.0134365615, 0.00786481, -9.82309284e-05, 0.00372130168, -0.0208442602, 0.0017130191, -0.0330817886, -0.0170152783, -0.0304350927, 0.0181460138, -0.0435261, 0.00824147835, -0.0295582283, 0.0211055037, 0.0223151147

In [165]:
res

{'ids': [['https://prod-img.thesouledstore.com/public/theSoul/uploads/catalog/product/1654844194_5240075.jpg',
   'https://prod-img.thesouledstore.com/public/theSoul/uploads/catalog/product/1654005818_5376730.jpg',
   'https://prod-img.thesouledstore.com/public/theSoul/uploads/catalog/product/1714500529_9949240.jpg',
   'https://prod-img.thesouledstore.com/public/theSoul/uploads/catalog/product/1695362629_1782444.jpg',
   'https://prod-img.thesouledstore.com/public/theSoul/uploads/catalog/product/1619684364_3680199.jpg']],
 'embeddings': None,
 'documents': [[None, None, None, None, None]],
 'uris': None,
 'data': None,
 'metadatas': [[None, None, None, None, None]],
 'distances': [[0.6596076488494873,
   0.7439491748809814,
   0.8050475716590881,
   0.8735591769218445,
   0.8888801336288452]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [79]:
import cv2
import os
from datetime import datetime

import cv2
def extract_frames(video_path, output_dir, frame_rate=1):
    """
    Extract frames from a video at a specified rate.

    Parameters:
    - video_path (str): Path to the video file.
    - output_dir (str): Directory to save the extracted frames.
    - frame_rate (int): Interval in seconds to capture frames.
    """
    # Create output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print("Error: Could not open video.")
        return

    # Get the frames per second (fps) of the video
    fps = int(video.get(cv2.CAP_PROP_FPS))
    frame_interval = int(fps * frame_rate)  # Interval in terms of frame count

    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break  # Break if video has ended

        # Save the frame at the specified interval
        if frame_count % frame_interval == 0:
            frame_path = os.path.join(output_dir, f"frame_{saved_count}.jpg")
            cv2.imwrite(frame_path, frame)
            saved_count += 1
            print(f"Saved frame {saved_count} at {frame_path}")

        frame_count += 1

    video.release()
    print("Frame extraction completed.")

# Usage example
out_dir = datetime.now().strftime("%d_%m_%Y")
os.mkdir(out_dir)
video_path = "/content/drive/MyDrive/Mumbai-Hackathon/videoplayback (1).mp4"  # Replace with your video path
output_dir = f"{out_dir}_output_frames"  # Directory to save frames
extract_frames(video_path, output_dir, frame_rate=20)  # Adjust frame_rate as needed


Saved frame 1 at 26_10_2024_output_frames/frame_0.jpg
Saved frame 2 at 26_10_2024_output_frames/frame_1.jpg
Saved frame 3 at 26_10_2024_output_frames/frame_2.jpg
Saved frame 4 at 26_10_2024_output_frames/frame_3.jpg
Saved frame 5 at 26_10_2024_output_frames/frame_4.jpg
Saved frame 6 at 26_10_2024_output_frames/frame_5.jpg
Saved frame 7 at 26_10_2024_output_frames/frame_6.jpg
Saved frame 8 at 26_10_2024_output_frames/frame_7.jpg
Saved frame 9 at 26_10_2024_output_frames/frame_8.jpg
Saved frame 10 at 26_10_2024_output_frames/frame_9.jpg
Frame extraction completed.


In [130]:
client_text = chromadb.PersistentClient(path="content")
collection_text = client.get_or_create_collection(name="content_db")

In [88]:
attr = ['product_id','product_name','product_type','description','neckline', 'sleeve_length', 'categories', 'artists', 'attributes', 'patterns', 'styles', 'material_compositions','occasions']

In [108]:
sem_data = []
for prod in prod_li:
  data = {}
  for a in attr:
    data[a] = prod[a]
  sem_data.append(data)

In [127]:
text_emb_final = []
model = TextEmbeddingModel.from_pretrained("text-embedding-004")

for batch in range(0,len(sem_data),15):
  dimensionality = 512
  task = "RETRIEVAL_DOCUMENT"
  batch_data = sem_data[batch:batch+15]
  inputs = [TextEmbeddingInput(str(text), task) for text in batch_data]
  kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}
  embeddings = model.get_embeddings(inputs, **kwargs)
  text_emb_li = [embedding.values for embedding in embeddings]
  text_emb_final.extend(text_emb_li)




In [132]:
id_li = [str(s.get("product_id")) for s in sem_data]

In [133]:
collection_text.add(
    ids = id_li,
    embeddings = text_emb_final
)

In [134]:
text = "I want rather I need clothes to wear in hackathon"

In [137]:
text = "I want rather I need clothes to wear in hackathon"

inputs = [TextEmbeddingInput(str(text), task)]
kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}
embeddings = model.get_embeddings(inputs, **kwargs)
text_emb_li = [embedding.values for embedding in embeddings]

result = collection_text.query(
    query_embeddings = text_emb_li[0],
    n_results = 5
)

In [139]:
result = collection_text.query(
    query_embeddings = text_emb_li[0],
    n_results = 5
)

In [140]:
result

{'ids': [['785364', '862463', '798026', '562769', '130325']],
 'embeddings': None,
 'documents': [[None, None, None, None, None]],
 'uris': None,
 'data': None,
 'metadatas': [[None, None, None, None, None]],
 'distances': [[0.4954566955566406,
   0.49559417366981506,
   0.4956825375556946,
   0.49677082896232605,
   0.49697571992874146]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [154]:
extr_feat

{'summary': ['a man wearing sunglasses and a light blue shirt with the words linen blend below him'],
 'background': ['linen blend', 'sailboat', 'ropes'],
 'clothes': ['linen blend',
  'linen blend white shirt black pants',
  'white shirt black pants'],
 'overall': ['casual', 'relaxed', 'cool']}

In [158]:
import os
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting, FinishReason
import vertexai.generative_models as generative_models
from vertexai.vision_models import ImageTextModel


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/hackathon-mumbai/aiml_creds.json"
vertexai.init(project="ai-coe-386307", location="us-central1")

def generate_vibes_query(data):

  model = GenerativeModel(
    "gemini-1.5-flash-001",
    system_instruction=['Your task is to generate a query explaining the vibe using image description. The resultant query will be used to extract products from database using semantic search.']
  )
  generation_config = {
      "max_output_tokens": 8192,
      "temperature": 1,
      "top_p": 0.95,
  }
  responses = model.generate_content(
      [f"Vibe description:\n{data}"],
      generation_config=generation_config
  )
  return responses.text


vibe_model = ImageTextModel.from_pretrained("imagetext@001")
image = Image.load_from_file("/content/drive/MyDrive/Mumbai-Hackathon/1716543492_6546785.webp")
extr_feat = {}
captions = vibe_model.get_captions(
    image=image,
    number_of_results=1,
    language="en",
)
extr_feat["summary"] = captions
prompts = [
    "Describe background of the image.",
    "Describe category and color of clothes in image.",
    "Describe overall vibe of the image."
]
ans = []
for p in prompts:
  answers = vibe_model.ask_question(
      image=image,
      question=p,
      number_of_results=3,
  )
  ans.append(answers)
extr_feat["background"] = [a if a!="unanserable" else "" for a in ans[0]]
extr_feat["clothes"] = [a if a!="unanserable" else "" for a in ans[1]]
extr_feat["overall"] = [a if a!="unanserable" else "" for a in ans[2]]


In [159]:
res = generate_vibes_query(extr_feat)

In [161]:
print(res)

## Query for Semantic Search:

**Products for a relaxed, cool, and casual summer look, featuring linen blend clothing, possibly with nautical or sailing themes.**

**Keywords:**

* Linen blend
* Casual
* Relaxed
* Cool
* Summer
* Nautical
* Sailing
* White shirt
* Black pants

**Explanation:**

This query combines the information from the vibe description to create a comprehensive search request. It highlights the key elements:

* **Fabric:** Linen blend is the most important characteristic, indicating a preference for lightweight and breathable fabrics.
* **Style:** Casual, relaxed, and cool define the desired aesthetic, suggesting comfortable and effortlessly stylish clothing.
* **Theme:** The background details (sailboat, ropes) hint at a potential nautical or sailing influence, which can be included as a secondary search criterion. 
* **Specific garments:** The mention of "white shirt black pants" provides additional detail for more precise product identification. 

By using these 